In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import numpy as np

In [ ]:
Qinfo = pd.read_csv('data/QInfo.csv')
Qinfo.head()

In [ ]:
Qbody = pd.read_csv('data/Qbody.csv')
Qbody

In [ ]:
Qbody.info()

In [ ]:
sample_html = Qbody['Body'][442]
sample_html

In [ ]:
soup = BeautifulSoup(sample_html, 'html.parser')
print(soup.text)

In [ ]:
def parse_question(question_html):
    soup = BeautifulSoup(question_html, 'html.parser')
    txt = soup.text.replace('\xa0', ' ').split('\n \n')[0]
    #match = re.search(r'(.+?\n\n)', txt, re.DOTALL)
    #txt = match.group(0)
    txt = txt.replace('-', ' ')
    txt = txt.replace('\n', ' ')
    return re.sub(r'[^\w\s]', '', txt).rstrip().lower()
parse_question(Qbody['Body'][0])

In [ ]:
Qbody['text'] = Qbody['Body'].apply(parse_question)

In [ ]:
vocab = set()
def update_set(txt):
    vocab.update(txt.split())
Qbody['text'].apply(update_set)
len(vocab)

In [ ]:
list(vocab)[:10]

In [ ]:
df = pd.DataFrame(
    {
        'word' : list(vocab)
    }
)
df.head()

In [ ]:
def term_freq_fnc(doc):
    def term_freq(word):
        doc_words = doc.split()
        return (doc_words.count(word)/len(doc_words))
    return term_freq

In [ ]:
term_freq = term_freq_fnc(Qbody['text'][0])
tmp_df = df['word'].apply(term_freq)

In [ ]:
tmp_df[tmp_df>0][:10]

In [ ]:
for i, doc in enumerate(Qbody['text']):
    term_freq = term_freq_fnc(doc)
    df[f'TF{i+1}'] = df['word'].apply(term_freq)

In [ ]:
no_of_docs = len(Qbody['text'])
no_of_docs

In [ ]:
tf_cols = [f'TF{i+1}' for i in range(no_of_docs)]
doc_freq = np.sum( (df[tf_cols] > 0).to_numpy(), axis=1, keepdims=True)
doc_freq

In [ ]:
idf = np.log(no_of_docs / doc_freq)
idf

In [ ]:
tf = df[tf_cols].to_numpy()
tf

In [ ]:
tf_idf = tf * idf

In [ ]:
tf_idf

In [ ]:
tmp = tf_idf.sum(axis=1)
np.where(tmp <= 0)

In [ ]:
df

In [ ]:
tfidf_DF = pd.DataFrame(
    tf_idf,
    columns=[f'TF_IDF{i+1}' for i in range(no_of_docs)]
)
tfidf_DF['word'] = df['word']
tfidf_DF

In [ ]:
tfidf_DF.iloc[438]

In [ ]:
tfidf_DF

In [ ]:
import numpy as np

# Create a sample 2D NumPy array
array_2d = np.array([
    [3, 1, 4, 1, 5, 9, 2, 6, 5, 3, 5, 8, 9, 7, 9, 3, 2, 3, 8, 4],
    [4, 2, 5, 7, 3, 1, 8, 3, 5, 6, 1, 9, 4, 2, 7, 8, 6, 1, 3, 5]
])

# Find the positions of the highest 20 values in each row
k = 2
highest_positions = []
for row in array_2d:
    partitioned_indices = np.argpartition(row, -k)[-k:]
    sorted_indices = partitioned_indices[np.argsort(row[partitioned_indices])]
    highest_positions.append(sorted_indices)

print(f"Positions of the highest {k} values in each row:")
for i, positions in enumerate(highest_positions):
    print(f"Row {i}: {positions}", [array_2d[i, p] for p in positions])
np.array(highest_positions)

In [ ]:
k = 20
highest_positions = []
for row in tf_idf:
    partitioned_indices = np.argpartition(row, -k)[-k:]
    sorted_indices = partitioned_indices[np.argsort(row[partitioned_indices])]
    highest_positions.append(sorted_indices)
highest_positions = np.array(highest_positions)

In [ ]:
highest_positions

In [ ]:
f"{(highest_positions.size * highest_positions.itemsize)//(1024)} kb"

In [ ]:
lookup_table = pd.DataFrame(
    highest_positions,
    columns=[f'DOC {i}' for i in range(20)]
)
lookup_table['word'] = df['word']
lookup_table

In [ ]:
searh_word = "apple"

idx = np.where((lookup_table['word'] == searh_word).to_numpy())[0]
lookup_table.iloc[idx]

In [ ]:
def map_to_QID(doc_no):
    return Qbody['QID'][doc_no]

In [ ]:
cols = [f'DOC {i}' for i in range(20)]
for col in cols:
    lookup_table[col] = lookup_table[col].apply(map_to_QID)
lookup_table

In [ ]:
searh_word = "apple"

idx = np.where((lookup_table['word'] == searh_word).to_numpy())[0]
if idx.size == 0:
    print('Not found')
else:
    idx = idx[0]
    doc_no = lookup_table['DOC 19'][idx]
    print(Qbody['text'][doc_no])
    print(Qbody['QID'][doc_no])
    print(Qinfo.iloc[doc_no - 1])

In [ ]:
# lookup table and Qinfo would be used by the app
Qinfo.to_csv('QINFO.csv', index=False, columns=Qinfo.columns[1:])
lookup_table.to_csv('LOOKUP.csv', index=False)